# INFO 2950 Final Project - Phase IV
##### Janice Shen (js3678), Khai Xin Kuan (kk996), Sandy Lin (sl2534), David Park (yp358)

## Introduction REVISE INTRO AFTER FINDINGS
*State your research question(s) clearly.*

*rubric: All expectations of typical projects + clearly describes why the setting is important and what is at stake in the results of the analysis. Citations to related literature are included that might clarify why the research questions are important, or introduce related research.
Even if the reader doesn’t know much about the subject, they know why they care about the results of your analysis.*

A common interest in both music and finance led the four of us to explore a unique intersection between these fields. Specifically, we asked: could individual music preferences be related to broader economic trends in the United States? Inspired by prior research that suggested promising correlations between music sentiment and market performance (e.g., weekly equity returns) , we decided to investigate further. While excited by the potential connections, our initial data analysis revealed only weak correlations between music sentiment and economic indicators. This prompted us to refine our approaches and think about regression models we could run for the next phase, hoping to reveal hidden trends. 

## Research Questions:
1. **Can the valence of the general population's music preference be predicted by economic indicators such as the unemployment rate, the federal funds rate, and other macroeconomic variables?**
   - *Valence* represents the spectrum of emotions in music, ranging from negative (0) to positive (1).
   - **Goal:** We sought to determine whether broader economic conditions influence the general emotional tone of music that people gravitate toward.

2. **How does danceability correlate with valence, and can we predict the danceability of the general population's music preference based on key economic variables?**
   - *Danceability* measures how suitable a track is for dancing, ranging from 0.0 (least danceable) to 1.0 (most danceable).
   - **Goal:** We explored whether this characteristic, often linked to musical enjoyment, also showed any connections to economic conditions.

### Initial Findings and Methodological Refinement:
Our original analysis included all songs in our dataset and yielded extremely weak correlations. We hypothesized that the inclusion of a large volume of less popular songs may have distorted the results, as these songs might not accurately reflect public sentiment or consumption patterns. To address this, we filtered out less popular songs using a *popularity* metric (a scale from 0 to 100, where 100 indicates the most popular songs). We created a cutoff of 70, focusing on the songs with popularity greater than 70 in our analysis to better capture the music that resonates with the general public.

**Goal:** Even after refining our data, we found weak correlations between music sentiment and the wider US economy. However, in future phases, we aim to obtain more meaningful insights into the relationship between music sentiments and economic variables, with the potential to reveal subtle but significant patterns. 

In [10]:
import pandas as pd
import numpy as np
import duckdb
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from sklearn.linear_model import LinearRegression, LogisticRegression
import statsmodels.api as sm
from statsmodels.api import OLS, add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor

## Data Collection and cleaning summary 
**Overall Summary:** The primary task was to match the Billboard song data with Spotify’s sentiment data. This was done by using song titles and artist names as common identifiers. Variations in how song names or artist names were spelled were resolved to make sure the match was accurate. Economic data(eg. Fed fund rate, unemployment rate) was collected through different government sources and merged all economic data into one dataset. Additional control variables were collected to provide context to the economic variables(eg, birth rate, death rate, recession) or other factors influencing music trends.

**Data Standardization:**
- Ensuring consistent formats for song titles and artist names in billboard and spotify dataset
- Ensuring economic variables were comparable across different datasets.
- Handling Missing Data: Missing values were identified and addressed by either excluding records or using imputation methods depending on the context of the missing data and its potential impact on analysis.

**Final Merged Dataset:** 
- After the cleaning and merging steps, a comprehensive, standardized dataset was created that included:
  - Billboard data (song rankings and deta)
  - Spotify sentiment data (emotional tone of songs)
  - Economic data (macroeconomic indicators)
  - Control variables (additional contextual factors).


Final dataset that contains a comprehensive overview of all the music sentiment information, economic information, and additional control variable factors 

In [38]:
final = pd.read_csv('final_clean_data.csv')

inspect the data and see the columns present in the table

In [40]:
final.head() 

,Unnamed: 0,danceability,valence,year,recession,election,fedfundrate,UnemploymentRate,US_GDP,Entertainment_GDP,year_1,State_prisons,fertility,DeathRate,S&P500
0,0,0.4500,0.3670,1997,0,0,5.46,4.941667,8362.655,37156.0,1997,1099347.0,1.971,877.7,0.33
1,1,0.6175,0.1140,1997,0,0,5.46,4.941667,8362.655,37156.0,1997,1099347.0,1.971,877.7,0.33
2,2,0.5500,0.0438,1997,0,0,5.46,4.941667,8362.655,37156.0,1997,1099347.0,1.971,877.7,0.33
3,3,0.6180,0.5610,1997,0,0,5.46,4.941667,8362.655,37156.0,1997,1099347.0,1.971,877.7,0.33
4,4,0.7150,0.6120,1997,0,0,5.46,4.941667,8362.655,37156.0,1997,1099347.0,1.971,877.7,0.33


Dropping columns not used in our analysis 

In [41]:
print(final.columns) 
final.drop(columns=['year_1','Unnamed: 0', 'Entertainment_GDP'], axis=1, inplace=True)
print(final.columns) 
final.head()

Index(['Unnamed: 0', 'danceability', 'valence', 'year', 'recession',
       'election', 'fedfundrate', 'UnemploymentRate', 'US_GDP',
       'Entertainment_GDP', 'year_1', 'State_prisons', 'fertility',
       'DeathRate', 'S&P500'],
      dtype='object')
Index(['danceability', 'valence', 'year', 'recession', 'election',
       'fedfundrate', 'UnemploymentRate', 'US_GDP', 'State_prisons',
       'fertility', 'DeathRate', 'S&P500'],
      dtype='object')


,danceability,valence,year,recession,election,fedfundrate,UnemploymentRate,US_GDP,State_prisons,fertility,DeathRate,S&P500
0,0.4500,0.3670,1997,0,0,5.46,4.941667,8362.655,1099347.0,1.971,877.7,0.33
1,0.6175,0.1140,1997,0,0,5.46,4.941667,8362.655,1099347.0,1.971,877.7,0.33
2,0.5500,0.0438,1997,0,0,5.46,4.941667,8362.655,1099347.0,1.971,877.7,0.33
3,0.6180,0.5610,1997,0,0,5.46,4.941667,8362.655,1099347.0,1.971,877.7,0.33
4,0.7150,0.6120,1997,0,0,5.46,4.941667,8362.655,1099347.0,1.971,877.7,0.33


Creating a dataframe that contains the average aggregate information for each year

In [37]:
average_final = duckdb.sql(""" 
                                SELECT
                                    Year,
                                    AVG(danceability) AS avg_danceability,
                                    AVG(valence) AS avg_valence,
                                    AVG(US_GDP) AS avg_gdp,
                                    AVG(UnemploymentRate) AS avg_unemployment,
                                    AVG(fedfundrate) AS avg_fedfundrate,
                                    AVG(State_prisons) AS avg_imprisonment,
                                    AVG(fertility) AS avg_fertility, 
                                    AVG(DeathRate) AS avg_deathrate, 
                                    AVG("S&P500") AS avg_stockreturn
                                FROM
                                    final
                                GROUP BY
                                    Year
                                ORDER BY
                                    Year
                                """).df()

average_final

,year,avg_danceability,avg_valence,avg_gdp,avg_unemployment,avg_fedfundrate,avg_imprisonment,avg_fertility,avg_deathrate,avg_stockreturn
0,1997,0.651218,0.549568,8362.655,4.941667,5.460000,1099347.0,1.9710,877.7,0.33
1,1998,0.630388,0.532448,8866.480,4.500000,5.353333,1152792.0,1.9990,870.1,0.28
2,1999,0.658871,0.566514,9411.682,4.216667,4.970000,1189806.0,2.0075,875.6,0.21
3,2000,0.678270,0.595720,10002.179,3.966667,6.235833,1209130.0,2.0560,869.0,-0.09
4,2001,0.644918,0.569649,10470.231,4.741667,3.887500,1208708.0,2.0305,858.8,-0.12








*rubric: All expectations of typical projects + credits and values data sources. Thoughtfully addresses whether the datasets are trustworthy and reliable as per Discussion 7.
Includes a comprehensive and clear discussion of different aspects of the data. The reader understands where the data comes from, why it’s useful for the analysis, and any relevant limitations. If needed, another data scientist could easily find the original data using this description and begin to replicate the analysis.
The data cleaning steps are clear and well-commented with markdown text comprehensively explaining the purpose for each step.*

## Data Description

#### What are the observations (rows) and the attributes (columns)?
**Observations**: Each row represents a song or an economic indicator for a specific time period. In the music dataset (musicdata), rows represent individual songs, while in the economic datasets (undata, econdata, us_gdp_data, us_entertainmentgdp_data), rows represent time intervals (e.g., months or quarters).  
**Attributes**: In the **music dataset**, the columns include variables like **valence** (a measure of positivity in music), **danceability**, **popularity**, **year**, and other sentiment-related metrics. For the **economic datasets** columns include **unemployment rate**, **GDP**, **federal funds rate**, and other economic indicators.


#### Why was this dataset created?  
The **music sentiment datasets** were created to explore the relationship between musical attributes (such as valence, danceability, and popularity) and factors like popularity or music trends over time. The **economic datasets** were generated by governmental agencies like the **Federal Reserve** to monitor key economic indicators for research, policy analysis, and public transparency.


#### Who funded the creation of the dataset?
The **music sentiment dataset** were compiled by independent researchers, such as Caleb Elgut, likely with no explicit funding. These data sources rely on platforms like **Spotify**, **Billboard**, and **ARIA**, which collected the original data. The **economic datasets** were funded by the U.S. government through organizations like the **Federal Reserve** and the **Federal Reserve Bank of St. Louis**, as part of their regular efforts to report and track national economic trends.


#### What processes might have influenced what data was observed and recorded and what was not?  
In the **music datasets**, factors like availability on Spotify, scraping techniques, and modern-day popularity metrics may have influenced the data observed. For example, older songs that were popular in the 1990s may have lower popularity today, potentially skewing the results when comparing them to historical economic data. In the **economic datasets**, revisions in how economic indicators like GDP and unemployment are calculated, government adjustments, and data collection methodologies could have impacted the recorded data. Additionally, the time series nature of the data means there may be adjustments due to seasonal variations or changes in reporting practices.


#### What preprocessing was done, and how did the data come to be in the form that you are using?  
- **Music datasets**: The data was scraped from platforms like **Spotify**. Preprocessing included cleaning missing values, and ensuring compatibility between different time periods in the analysis (there was no missing values when we loaded the files).  
- **Economic datasets**: The raw data provided by the **Federal Reserve** was merged and aligned with the music data for time-series analysis. Preprocessing involved cleaning missing values, ensuring consistent date formats, and aligning economic metrics with the appropriate historical periods for comparison with music data.


#### If people are involved, were they aware of the data collection and if so, what purpose did they expect the data to be used for?
- **Music datasets**: The data originates from public platforms like **Spotify**, where users are generally aware that their listening habits are tracked. However, artists and listeners would likely be unaware of this specific usage of the data for correlating music preferences with economic trends.  
- **Economic datasets**: Data on economic indicators is collected by government agencies like the **Federal Reserve**, with the understanding that it will be publicly available for research, policy analysis, and economic forecasting.


#### Where can your raw source data be found, if applicable?
1. **Music sentiment datasets**:  
   - Caleb Elgut's dataset scraped from Spotify can be found on GitHub: [Spotify-LSTM Data](https://github.com/calebelgut/spotify-lstm/tree/main/Data)  


2. **Economic datasets**:  
   - Abigail Larion’s dataset on economic indicators (including federal funds rate, real GDP, and unemployment rate) can be found on Kaggle: [Interest Rates Dataset](https://www.kaggle.com/datasets/federalreserve/interest-rates)  
   - Unemployment rate data from Jan 1948 – Sep 2024 is available through the Federal Reserve Economic Data (FRED) platform: [Unemployment Rate](https://fred.stlouisfed.org/series/UNRATE)  
   - Quarterly GDP from Jan 1997 – 2019: [Quarterly GDP](https://fred.stlouisfed.org/series/GDP)  
   - Yearly GDP of the Music/Entertainment Industry from 1997 – 2019: [Entertainment Industry GDP](https://fred.stlouisfed.org/series/USPRFRMSPRTMSMNGSP)

## Preregistration statement
*Provide at least 2 preregistration statements. Each statement contains a hypothesis and a description of analysis to study the hypothesis. It should also include context for why you believe in your hypothesis
by referencing domain knowledge or other literature. Refer to Phase 3 rubric for
more details.*

*rubric: At least 2 statements are detailed. Each statement includes 1 hypothesis and 1 description of the analysis. The preregistered analyses reflect deep and critical thinking about the real-world application from which the data originates. The hypothesis is substantiated by a description of domain knowledge and likely includes some citations. The analyses are described in a way that persuades the reader that their results will be interesting, whether or not they turn out to be
statistically significant.
The analyses referenced are scientifically valid and compelling.*

Hypothesis 1: The average valence of popular songs by year from 1960-2016 in the US has an inverse relationship with the corresponding US Gross Domestic Product (GDP) of the given year.

Analysis: 
We plan to run a linear regression where we input the US GDP (which is the total value of final goods and services of the country and thus indicates an aspect of the US economy) and output the average valence (a measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track) of each year’s popular (scraped from Billboard top 100) songs. Therefore, we expect to see β<0. Based on our prior EDA, there is a negative correlation between the US GDP and valence. 


Hypothesis 2: The average danceability of the most popular songs per year from 1960-2016 in the US has an inverse relationship with corresponding US unemployment rate of the given year.

Analysis: 
We will run a linear regression where the input is the average danceability (a measure from 0.0 to 1.0 describing how suitable a track is for dancing based on a combination of musical elements including tempo and rhythm) of each year’s popular songs (scraped from Billboard’s top 100 songs), and output the unemployment rate, which is the percentage of unemployed people actively seeking for jobs. Since we are trying to test the inverse relationship, we expect to see that β < 0. Based on our prior EDA, there is a negative correlation between the US unemployment rate and danceability. 
__________-^notes from phase 3
### First Research Question: Music Valence vs US GDP
**The average valence of popular songs by year from 1960-2016 in the US has an inverse relationship with the corresponding US Gross Domestic Product (GDP) of the given year.** 
- $\beta$ < 0
- $H_0$: The valence of the top 100 Spotify songs per year has no relationship to the US GDP each year.
- $H_A$: The valence of the top 100 Spotify songs per year has an inverse relationship to the US GDP each year.

### Second Research Question: Music Danceability vs US Unemployment
**The average danceability of the most popular songs per year from 1960-2016 in the US has an inverse relationship with corresponding US unemployment rate of the given year.** 
- $\beta$ < 0
- $H_0$: The danceability of the top 100 Spotify songs per year has no relationship to the US unemployment each year.
- $H_A$: The danceability of the top 100 Spotify songs per year has an inverse relationship to the US unemployment each year.

## Data analysis
Carry out the analysis for each preregistration statement.
Interpret analyses appropriately (e.g., regressions should be interpreted using the
summarizing, predicting, and outliers/oddities framework from class).
rubric: All expectations of typical projects + analyses are carefully selected to answer all reasonable questions.
Questions raised by one analysis are addressed in subsequent analyses.
There is not “analysis barf”: i.e., analyses are chosen to clearly answer the research questions. All analyses are interpreted clearly (i.e., there is markdown text describing what is being done, and what the result is, for each analysis).

## Evaluation of significance
Determine statistical significance of observed data.
Interpret findings and conclude whether to reject or fail to reject hypotheses
outlined in preregistration statements.
rubric: Metrics of statistical significance appropriate to the analysis performed are presented. Description of metrics / interpretations of discovered p-values are correct and clear.
Numeric values are interpreted in a way that supports a clear story and leads to a conclusion that creatively ties the analysis together. Overall, presents the results of the analysis through a well-written and cohesive discussion.
Values are presented in context and with clear labels.
No “chart barf”. All figures include an explanation as to why they are shown and interpretations for each figure or table.

## Conclusions
Summarize your answers to your opening research questions.
rubric: Research questions are answered clearly and concisely, using rigorous findings from the previous sections to back up claims.
Substantively engages with the data analysis and the evaluation of significance sections.
Uses key findings from these sections as evidence to support claims in the conclusion.

## Limitations
Outline any potential harms and data gaps and describe how this
could affect conclusions made.
rubric: Creatively identifies potential harms and data gaps, and describes how these could affect the meaning of results, as well as the impact of results on people. Limitations of significance metrics are acknowledged.

## Acknowledgements and Bibliography
Think of this as your Problem 0: what
tools, data, or resources did you use that you should cite? You may format this
section with any citation style.
rubric: Clearly marked and formatted. The group thoughtfully engages with relevant literature on their topic in order to motivate and/or contextualize their findings. All external sources are properly cited in the bibliography and with appropriate in-text citations.